Create Azure SQL Database
============================================

Steps of this procedure include:

1. Connect to Azure subscription
1. Provision resource group for SQL Managed Instance
1. Create Sql Server
1. Provision firewall rules to allow access
1. Create Sql Database

## Notebook Setup 

In [ ]:
import sys, os, json, time, string, random, subprocess
def run_command(command, json_decode = True, printOutput = True):
    print(command)
    process = subprocess.Popen(command.split(), stdout=subprocess.PIPE, stderr=subprocess.PIPE, shell=True)
    output, error = process.communicate()
    if process.returncode != 0: 
        print("Process failed %d \n%s" % (process.returncode, error.decode("utf-8")))
        raise Exception()
    if output:
        output = output.decode("utf-8")
        if printOutput:
            print(output)
        try:
            return json.loads(output)
        except:
            return output  return output

def get_random_string(length):
    letters = string.ascii_lowercase
    result_str = ''.join(random.choice(letters) for i in range(length))
    print("Random string of length", length, "is:", result_str)
    return result_str

## Set variables

In [ ]:
if "AZDATA_NB_VAR_AZURE_SQLDB_SERVER_PASS" in os.environ:
    azure_sqldb_password = os.environ["AZDATA_NB_VAR_AZURE_SQLDB_SERVER_PASS"]

## Connect to Azure Account

In [ ]:
subscriptions = run_command('az account list', printOutput = False)
if azure_sqldb_nb_var_subscription not in (subscription["id"] for subscription in subscriptions):
    run_command('az login')

## Set Subscription.

In [ ]:
run_command(
    'az account set '
    '--subscription {0}'
    .format(
        azure_sqldb_nb_var_subscription));

## Create the New Resource Group

In [ ]:
resource_group = run_command('az group exists --name {0}'
.format(azure_sqldb_nb_var_resource_group_name))


if (! resource_group):
    create_resource_group_result = run_command(
        'az group create '
        '--location {0} '
        '--name {1} '
        .format(
            azure_sqldb_location, 
            azure_sqldb_resource_nb_var_resource_group_name));

## Create a SQL server with a system wide unique server name

In [ ]:
server_name = run_command('az sql server show --name {0}'
.format(azure_sqldb_server_name))



if (! server_name):
    create_server_result = run_command(
        'az sql server create '
        '--location {0} '
        '--name {1} '
        '--admin-username {2} '
        '--admin-password {3} '
        '--resource-group {4} '
        .format(
            azure_sqldb_location, 
            azure_sqldb_server_name, 
            azure_sqldb_server_username, 
            azure_sqldb_password, 
            azure_sqldb_resource_nb_var_resource_group_name));

## Create a server firewall rule that allows access from the specified IP range

In [ ]:
    create_firewall_rule_result = run_command(
        'az sql server firewall-rule create '
        '--start-ip-address {0} '
        '--end-ip-address {1} '
        '--server {2} '
        '--name {3} '
        '--admin-username {4} '
        '--admin-password {5} '
        '--resource-group {6} '
        .format(
            azure_sqldb_ip_start, 
            azure_sqldb_ip_end, 
            azure_sqldb_server_name, 
            azure_sqldb_server_username, 
            azure_sqldb_password, 
            azure_sqldb_resource_nb_var_resource_group_name));

## Create SQL Database

In [ ]:
    create_database_result = run_command(
        'az sql db create '
        '--server {1} '
        '--name {2} '
        '--edition GeneralPurpose '
        '--capacity 2 '
        '--family Gen5 '
        '--min-capacity 2 '
        '--resource-group {3} '
        .format(
            azure_sqldb_server_name, 
            azure_sqldb_database_name, 
            azure_sqldb_resource_nb_var_resource_group_name));

## Get Database details

In [ ]:
    show_database_result = run_command(
        'az sql db show '
        '--server {1} '
        '--name {2} '
        '--resource-group {3} '
        .format(
            azure_sqldb_server_name, 
            azure_sqldb_database_name, 
            azure_sqldb_resource_nb_var_resource_group_name));